# Examine data and test

In [31]:
import pandas as pd

df_full = pd.read_csv("data/ciudad_autonoma_de_buenos_aires.csv",sep=";")


In [42]:

df["Agrupacion"].unique()
# df["Cargo"].unique()
df_full["Seccion"].unique()
# df["tipoVoto"].unique()


array(['Comuna 1', 'Comuna 4', 'Comuna 3', 'Comuna 7', 'Comuna 5',
       'Comuna 2', 'Comuna 10', 'Comuna 8', 'Comuna 13', 'Comuna 6',
       'Comuna 11', 'Comuna 15', 'Comuna 12', 'Comuna 9', 'Comuna 14'],
      dtype=object)

In [30]:
for agrupacion in df['Agrupacion'].unique():
    print(agrupacion)

    df_agrupacion  = df.loc[df["Agrupacion"] == agrupacion]
    # print(df_agrupacion["tipoVoto"].unique())
    print(df_agrupacion["votos"].sum())
    print()

FEDERAL
16810

CONSERVADOR POPULAR
7054

MOVIMIENTO AL SOCIALISMO
25627

CORRIENTE DE PENSAMIENTO BONAERENSE
4278

VERDE
16452

POPULAR
12517

TODOS POR BUENOS AIRES
17547

FRENTE PATRIOTA
19274

REPUBLICANO FEDERAL
28971

MORAL Y PROGRESO
960

VOCACION SOCIAL
34742

UNIDAD SOCIAL
2970

CELESTE PROVIDA
16605

LABORISTA
875

POLITICA OBRERA
11577

JUSTICIA Y DIGNIDAD PATRIOTICA
3117

ESPERANZA DEL PUEBLO
5143

+ VALORES
44264

PROYECTO JUSTO, SOCIAL Y HUMANISTA
8662

AVANZA LIBERTAD
156645

FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD
162868

FRENTE UNION POR EL FUTURO
32753

JUNTOS
1098696

FRENTE DE TODOS
964773

FRENTE VAMOS CON VOS
97899



# Import and clean data

In [58]:
import pandas as pd


# df_full = pd.read_csv("data/Buenos Aires_ Primera.csv",sep=";")
df_full = pd.read_csv("data/ciudad_autonoma_de_buenos_aires.csv",sep=";")

In [59]:
df = df_full.loc[df_full["Cargo"] == "DIPUTADOS NACIONALES"] # Filter for only diputados
df = df[~df["Agrupacion"].isnull()] # Remove nan

In [3]:
df["tipoVoto"].unique()
df["Agrupacion"].unique()

parties_left = ["POLÍTICA OBRERA", "SOCIALISTA AUTÉNTICO", "FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD", "MOVIMIENTO AL SOCIALISMO"]

# Socialista autentico
# Movimiento al socialismo
# Frente de izquierda 
# Politica obrera

# Heatmap

In [60]:
comunas = df["Seccion"].unique()

result_percentage= {}

for comuna in comunas:
    df_comuna = df.loc[df["Seccion"] == comuna]
    df_left = df_comuna.loc[ df_comuna["Agrupacion"].isin(parties_left) ]
    df_rest = df_comuna.loc[ ~df_comuna["Agrupacion"].isin(parties_left) ]

    votes_total = df_comuna["votos"].sum()
    votes_left = df_left["votos"].sum()
    votes_rest = df_rest["votos"].sum()
    
    result_percentage[comuna] = votes_left/votes_total


for comuna, percent in result_percentage.items():
    print(f"{comuna}: {percent}")

Comuna 1: 0.07424522775665046
Comuna 4: 0.08575243600469877
Comuna 3: 0.08700430714391802
Comuna 7: 0.07990550072885076
Comuna 5: 0.09462022064661978
Comuna 2: 0.04981461806985972
Comuna 10: 0.07996922095227371
Comuna 8: 0.07729854738838976
Comuna 13: 0.06546221178479932
Comuna 6: 0.08005288673424416
Comuna 11: 0.07476605173239402
Comuna 15: 0.09562928414953006
Comuna 12: 0.07812604325086242
Comuna 9: 0.0751456329563247
Comuna 14: 0.061687583637744584


In [61]:
# Find min and max
percent_max = 0
percent_min = 1
for key, percent in result_percentage.items():
    if percent > percent_max:
        percent_max = percent

    if percent < percent_min:
        percent_min = percent
    

# Map linearly to span[255*percentage_min,255], highest value goes to 0 (=most red)
val_max = 255
val_min = 255*percent_min

k = (val_max - val_min) / (percent_max - percent_min)
m = val_max - k * percent_max

result_255 = {}
for key, percent in result_percentage.items():
    value = k*percent + m
    result_255[key] = int(255-value) 


result_hex = {}
for key, integer in result_255.items():
    
    key_integer = key.split()[1]
    
    result_hex[key_integer] = f"ff{integer:02x}{integer:02x}"
    

In [62]:
# Create new svg from template file
import re
with open("images/comunas_template.svg", 'r') as f_template:

    text = f_template.read()
    
    for key, hex_val in result_hex.items():
        template_val = f"\\{{{key}\\}}"
        text = re.sub(template_val, hex_val, text)

    with open('images/comunas_heatmap.svg', 'w') as f_output:
        f_output.write(text)
    # f.seek(0) # Go to start
    # f.truncate() # Fix new file size(?)

# Heatmap Provinces

In [54]:
import pandas as pd
parties_left = ["POLÍTICA OBRERA", "SOCIALISTA AUTÉNTICO", "FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD", "MOVIMIENTO AL SOCIALISMO"]

data_directory = "data/provinces/"

percentages_provinces = {}


for filename in os.listdir(data_directory):
    
    # Read and clean file
    df_province = pd.read_csv(f"{data_directory}{filename}", sep=";")
    df_province = df_province.loc[df_province["Cargo"] == "DIPUTADOS NACIONALES"] # Filter for only diputados
    df_province = df_province.loc[~df_province["Agrupacion"].isnull()] # Remove nan
    
    print(filename)
    # Get only leftist parties
    df_left = df_province.loc[ df_province["Agrupacion"].isin(parties_left) ]
    
    # Get votes
    votes_total = df_province["votos"].sum()
    votes_left = df_left["votos"].sum()
    

    # Calculation
    percentage = votes_left/votes_total

    # Insert result in dictionary
    name_province = filename.split('.')[0]
    percentages_provinces[name_province] = percentage


Chubut.csv
Mendoza.csv
Formosa.csv
La_Rioja.csv
San_Juan.csv
San_Luis.csv
ciudad_autonoma_de_buenos_aires.csv
Jujuy.csv
Santa_Fe.csv
Catamarca.csv
Rio_Negro.csv
Tierra_del_Fuego,_Antаrtida_e_Islas_del_Atlаntico_Sur.csv
Neuguen.csv
Entre_Rios.csv
Misiones.csv
Cordoba.csv
Santa_Cruz.csv
Salta.csv
La_Pampa.csv
Santiago_del_Estero.csv
Tucuman.csv
Corrientes.csv
Chaco.csv


In [55]:
# Find min and max
percent_max = 0
percent_min = 1
for key, percent in percentages_provinces.items():
    if percent > percent_max:
        percent_max = percent

    if percent < percent_min:
        percent_min = percent
    

# Map linearly to span[255*percentage_min,255], highest value goes to 0 (=most red)
val_max = 255
val_min = 255*percent_min

k = (val_max - val_min) / (percent_max - percent_min)
m = val_max - k * percent_max

result_255 = {}
for key, percent in percentages_provinces.items():
    value = k*percent + m
    result_255[key] = int(255-value) 


result_hex = {}
for key, integer in result_255.items():
    result_hex[key] = f"ff{integer:02x}{integer:02x}"

In [56]:
# Create new svg from template file
import re
with open("images/argentina_provinces_template.svg", 'r') as f_template:

    text = f_template.read()
    
    for key, hex_val in result_hex.items():
        template_val = f"\\{{{key}\\}}"
        text = re.sub(template_val, hex_val, text)

    with open('images/provinces_heatmap.svg', 'w') as f_output:
        f_output.write(text)
    # f.seek(0) # Go to start
    # f.truncate() # Fix new file size(?)

In [53]:
result_hex["Tucuman"]

KeyError: 'Tucuman'

In [42]:
result_hex["Tucumаn"]

'ffdcdc'

In [14]:
import pandas as pd
parties_left = ["POLÍTICA OBRERA", "SOCIALISTA AUTÉNTICO", "FRENTE DE IZQUIERDA Y DE TRABAJADORES - UNIDAD", "MOVIMIENTO AL SOCIALISMO"]
# Test with data
df_provinces_full = pd.read_csv("data/provinces/Formosa.csv",sep=";")
df_prov = df_provinces_full.loc[df_provinces_full["Cargo"] == "DIPUTADOS NACIONALES"] # Filter for only diputados
df_prov = df_prov[~df_prov["Agrupacion"].isnull()] # Remove nan

In [16]:
df_prov

,Agrupacion,Cargo,Codigo,Distrito,Establecimiento,Fecha,IdCargo,IdCircuito,IdDistrito,IdSeccion,Mesa,Seccion,idAgrupacion,idAgrupacionInt,idLista,lista,tipoVoto,votos\t
0,FRENTE DE TODOS,DIPUTADOS NACIONALES,60,Formosa,EPEP N° 335,12-09-2021 22:52,3,0005A,9,1,00442X,Formosa,501.0,145.0,1.0,CELESTE\tBLANCA Y AZUL,positivo,169
1,JUNTOS POR FORMOSA LIBRE,DIPUTADOS NACIONALES,60,Formosa,EPEP N° 335,12-09-2021 22:52,3,0005A,9,1,00442X,Formosa,502.0,146.0,1.0,JUNTOS POR EL CAMBIO,positivo,29
2,FRENTE ESTAMOS CON VOS,DIPUTADOS NACIONALES,60,Formosa,EPEP N° 335,12-09-2021 22:52,3,0005A,9,1,00442X,Formosa,503.0,147.0,1.0,SIEMPRE CON VOS,positivo,48
8,PRINCIPIOS Y CONVICCIÓN,DIPUTADOS NACIONALES,61,Formosa,EPEP N° 378,12-09-2021 19:46,3,00006,9,1,00443X,Formosa,212.0,144.0,1.0,ESPERANZA,positivo,7
9,FRENTE DE TODOS,DIPUTADOS NACIONALES,61,Formosa,EPEP N° 378,12-09-2021 19:46,3,00006,9,1,00443X,Formosa,501.0,145.0,1.0,CELESTE\tBLANCA Y AZUL,positivo,98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12815,FRENTE ESTAMOS CON VOS,DIPUTADOS NACIONALES,305,Formosa,EPEP Nº 355,12-09-2021 23:54,3,0084C,9,9,01440X,Ramón Lista,503.0,147.0,1.0,SIEMPRE CON VOS,positivo,6
12821,PRINCIPIOS Y CONVICCIÓN,DIPUTADOS NACIONALES,305,Formosa,EPEP Nº 355,12-09-2021 23:56,3,0084C,9,9,01441X,Ramón Lista,212.0,144.0,1.0,ESPERANZA,positivo,2
12822,FRENTE DE TODOS,DIPUTADOS NACIONALES,305,Formosa,EPEP Nº 355,12-09-2021 23:56,3,0084C,9,9,01441X,Ramón Lista,501.0,145.0,1.0,CELESTE\tBLANCA Y AZUL,positivo,67
12823,JUNTOS POR FORMOSA LIBRE,DIPUTADOS NACIONALES,305,Formosa,EPEP Nº 355,12-09-2021 23:56,3,0084C,9,9,01441X,Ramón Lista,502.0,146.0,1.0,JUNTOS POR EL CAMBIO,positivo,100
